In [ ]:
from ultralytics import YOLO
import cv2
import csv
import torch
from super_image import EdsrModel
import numpy as np
import easyocr
from skimage.filters import threshold_sauvola

# --- CONFIGURACIÓN ---
VIDEO_SOURCE = "C0142.MP4"
VEHICLE_MODEL = "yolo11n.pt"
PLATE_MODEL = "best.pt"
CONF_VEHICLE = 0.35
CONF_PLATE = 0.5
DEVICE = "cuda"

# --- MODELOS ---
vehicle_model = YOLO(VEHICLE_MODEL)
plate_model = YOLO(PLATE_MODEL)

# OCR
reader = easyocr.Reader(['es'], gpu=True, detect_network='craft', recog_network='standard')

# Super resolución
super_resolution_model = EdsrModel.from_pretrained('eugenesiow/edsr', scale=4).to('cuda')

# --- VIDEO DE ENTRADA/SALIDA ---
cap = cv2.VideoCapture(VIDEO_SOURCE)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

human_count = 0
vehicle_count = 0
tracked_ids = set()

out = cv2.VideoWriter(
    "salida.mp4",
    cv2.VideoWriter_fourcc(*'mp4v'),
    fps,
    (frame_width, frame_height)
)

# --- CSV ---
csv_file = open("detecciones.csv", mode='w', newline='')
csv_writer = csv.writer(csv_file, delimiter=';')
csv_writer.writerow([
    "fotograma", "tipo_objeto", "confianza", "identificador_tracking",
    "x1", "y1", "x2", "y2",
    "matricula_detectada", "confianza_matricula",
    "mx1", "my1", "mx2", "my2", "texto_matricula"
])


# --- FUNCIONES ---
def enhance_for_ocr(img):
    """
    Mejora la imagen antes del OCR aplicando realce de luminancia,
    reducción de ruido y umbralizado adaptativo híbrido (CLAHE + Sauvola).
    Devuelve una imagen binaria limpia y nítida lista para EasyOCR.
    """
    if img is None or not isinstance(img, np.ndarray) or img.size == 0:
        return np.zeros((1, 1), dtype=np.uint8)

    # 1️⃣ Ecualización suave de luminancia (en color)
    ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    y, cr, cb = cv2.split(ycrcb)
    y = cv2.equalizeHist(y)
    ycrcb = cv2.merge([y, cr, cb])
    img_eq = cv2.cvtColor(ycrcb, cv2.COLOR_YCrCb2BGR)

    # 2️⃣ Suavizado y realce leve
    img_eq = cv2.bilateralFilter(img_eq, 5, 50, 50)
    blur = cv2.GaussianBlur(img_eq, (0, 0), 1.0)
    sharpened = cv2.addWeighted(img_eq, 1.3, blur, -0.3, 0)

    # 3️⃣ Conversión a escala de grises
    gray = cv2.cvtColor(sharpened, cv2.COLOR_BGR2GRAY)

    # 4️⃣ Aumento local de contraste (CLAHE)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    clahe_gray = clahe.apply(gray)

    # 5️⃣ Umbral Sauvola (robusto en sombras)
    try:
        sauvola_thresh = threshold_sauvola(clahe_gray, window_size=25, k=0.3)
        binary_sauvola = (clahe_gray > sauvola_thresh).astype(np.uint8) * 255
    except Exception:
        binary_sauvola = clahe_gray  # fallback si skimage falla

    # 6️⃣ Umbral Otsu global
    _, binary_otsu = cv2.threshold(
        clahe_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU
    )

    # 7️⃣ Combinación híbrida (OR lógico)
    combined = cv2.bitwise_or(binary_sauvola, binary_otsu)

    # 8️⃣ Corrección de inversión si el texto es oscuro
    if np.mean(combined) < 128:
        combined = cv2.bitwise_not(combined)

    # 9️⃣ Suavizado morfológico final
    kernel = np.ones((2, 2), np.uint8)
    combined = cv2.morphologyEx(combined, cv2.MORPH_CLOSE, kernel, iterations=1)

    # 🔟 Rellenar huecos internos
    kernel_close = np.ones((3, 3), np.uint8)
    closed = cv2.morphologyEx(combined, cv2.MORPH_CLOSE, kernel_close, iterations=2)

    # Invertir para rellenar agujeros
    inv = cv2.bitwise_not(closed)
    contours, _ = cv2.findContours(inv, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        cv2.drawContours(inv, [cnt], 0, 255, -1)

    filled = cv2.bitwise_not(inv)
    return filled


# --- PROCESAMIENTO PRINCIPAL ---
frame_id = 0
best_plate_match = {}

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_id += 1
    clean_frame = frame.copy()

    results = vehicle_model.track(
        frame, persist=True, classes=[0, 2], conf=CONF_VEHICLE, device=DEVICE
    )

    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy.cpu().numpy()[0])
            conf = round(float(box.conf[0]), 2)
            cls = int(box.cls[0])
            name = vehicle_model.model.names.get(cls, str(cls))
            track_id = int(box.id[0]) if box.id is not None else -1

            has_plate = 0
            plate_conf = 0
            mx1 = my1 = mx2 = my2 = ''
            plate_text = ''

            # Solo coches tienen matrícula
            if name == "car":
                print(f"Procesando frame {frame_id}/{total_frames} ({frame_id / total_frames * 100:.1f}%)", end="\r")

                crop = clean_frame[y1:y2, x1:x2]
                if len(crop.shape) == 2:
                    crop = cv2.cvtColor(crop, cv2.COLOR_GRAY2BGR)

                plate_results = plate_model.predict(
                    crop, conf=CONF_PLATE, device=DEVICE
                )

                for pr in plate_results:
                    for pbox in pr.boxes:
                        has_plate = 1
                        px1, py1, px2, py2 = map(int, pbox.xyxy.cpu().numpy()[0])
                        plate_conf = round(float(pbox.conf[0]), 2)

                        mx1, my1, mx2, my2 = (
                            x1 + px1,
                            y1 + py1,
                            x1 + px2,
                            y1 + py2,
                        )

                        plate_img = clean_frame[my1:my2, mx1:mx2]
                        if plate_img.size == 0:
                            continue

                        # --- Superresolución ---
                        img_rgb = cv2.cvtColor(plate_img, cv2.COLOR_BGR2RGB)
                        img_tensor = (
                            torch.from_numpy(img_rgb)
                            .permute(2, 0, 1)
                            .float()
                            .div(255.0)
                            .unsqueeze(0)
                            .to(DEVICE)
                        )

                        with torch.inference_mode():
                            preds = super_resolution_model(img_tensor)

                        out_tensor = preds[0].clamp(0, 1).cpu()
                        out_array = (
                            out_tensor.permute(1, 2, 0).numpy() * 255
                        ).astype("uint8")

                        # --- Mejora y OCR ---
                        plate_processed = enhance_for_ocr(out_array)
                        res = reader.readtext(
                            plate_processed,
                            allowlist="ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789",
                            paragraph=False,
                        )

                        best_plate_match.setdefault(track_id, ("", 0.0))
                        current_text = " ".join([t for (_, t, _) in res]) if res else ""
                        current_text += f" conf({plate_conf})"
                        current_conf = max(
                            [float(c) for (_, _, c) in res], default=0.0
                        )

                        # Guardar mejor lectura

                        if current_conf + len(current_text) * 0.05 > best_plate_match[track_id][1] + len(best_plate_match[track_id][0]) * 0.05:
                            best_plate_match[track_id] = (current_text, current_conf)

                        plate_text, plate_conf = best_plate_match[track_id]

                        # Dibujar caja verde
                        cv2.rectangle(
                            frame, (mx1, my1), (mx2, my2), (0, 255, 0), 2
                        )
                        cv2.putText(
                            frame,
                            plate_text,
                            (mx1, my1 - 5),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.6,
                            (0, 255, 0),
                            2,
                        )
                        h, w = blur_plate.shape[:2]
                        blur_plate = cv2.resize(cv2.resize(blur_plate, (w // 10, h // 10)), (w, h), interpolation=cv2.INTER_NEAREST)
                        # Resize blur_plate to match the target region's shape
                        blur_plate_resized = cv2.resize(blur_plate, (mx2 - mx1, my2 - my1), interpolation=cv2.INTER_NEAREST)
                        frame[my1:my2, mx1:mx2] = blur_plate_resized
                        break

            # --- Dibujar caja principal ---
            if name == "person":
                # blur 1/4 original person area
                blur_face = cv2.GaussianBlur(frame[y1:y2 + 3 * (y1 - y2) // 4, x1:x2], (51, 51), 0)
                frame[y1:y2 + 3 * (y1 - y2) // 4, x1:x2] = blur_face
                if track_id not in tracked_ids:
                    tracked_ids.add(track_id)
                    human_count += 1
                color = (255, 100, 0)
            else:
                if track_id not in tracked_ids:
                    tracked_ids.add(track_id)
                    vehicle_count += 1
                color = (0, 100, 255)
                
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(
                frame,
                f"{track_id} {name}",
                (x1, y1 - 5),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.7,
                (255, 0, 0),
                2,
            )

            # --- Escribir CSV ---
            csv_writer.writerow([
                frame_id, name, conf, track_id,
                x1, y1, x2, y2,
                has_plate, plate_conf,
                mx1, my1, mx2, my2, plate_text
            ])

    out.write(frame)

# --- LIMPIEZA FINAL ---
cap.release()
out.release()
csv_file.close()

print("\nVídeo 'salida.mp4' y CSV 'detecciones.csv' generados correctamente.")
print("se detectaron un total de", human_count, "personas y", vehicle_count, "vehículos.")


https://huggingface.co/eugenesiow/edsr/resolve/main/pytorch_model_4x.pt

0: 384x640 2 cars, 11.2ms
Speed: 34.9ms preprocess, 11.2ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
Procesando frame 1/2832 (0.0%)
0: 320x416 (no detections), 12.9ms
Speed: 0.6ms preprocess, 12.9ms inference, 0.5ms postprocess per image at shape (1, 3, 320, 416)
Procesando frame 1/2832 (0.0%)
0: 288x416 (no detections), 14.0ms
Speed: 0.6ms preprocess, 14.0ms inference, 0.8ms postprocess per image at shape (1, 3, 288, 416)

0: 384x640 2 cars, 12.6ms
Speed: 1.4ms preprocess, 12.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
Procesando frame 2/2832 (0.1%)
0: 352x416 (no detections), 10.7ms
Speed: 0.6ms preprocess, 10.7ms inference, 0.6ms postprocess per image at shape (1, 3, 352, 416)
Procesando frame 2/2832 (0.1%)
0: 256x416 (no detections), 11.4ms
Speed: 0.8ms preprocess, 11.4ms inference, 0.8ms postprocess per image at shape (1, 3, 256, 416)

0: 384x640 2 cars, 10.6ms
